(sec:exercise-othello)=
# 演習3 - オセロエージェントを作る

今回の演習では、いわゆるオセロのAIを作成してみよう。

```{warning}
オセロとは日本で生まれた「リバーシ」の一種であり、リバーシとオセロには、ルールの差こそないものの、若干の違いがある。これらの違いについてはWikipedia等を参照のこと。

参考: [オセロ（ボードゲーム）- Wikipedia](https://ja.wikipedia.org/wiki/%E3%82%AA%E3%82%BB%E3%83%AD_(%E3%83%9C%E3%83%BC%E3%83%89%E3%82%B2%E3%83%BC%E3%83%A0))
```

## 演習内容

今回の演習では**オセロの盤面を表す二次元配列**が与えられた時に、次の手を返すようなAIを作成する。

In [2]:
from typing import Tuple

import numpy as np
import numpy.typing as npt


def play(board: npt.NDArray[np.int32]) -> Tuple[int, int]:
    pass

**入力の例**

```python
board = [
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
]
```